In [2]:
!pip install -U google-generativeai
!pip install genai
!pip install -U langchain-community
!pip install faiss-cpu
!pip install camel-tools

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import camel_tools 
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_teh_marbuta_ar
from camel_tools.utils.dediac import dediac_ar
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator
import re
from camel_tools.tokenizers.word import simple_word_tokenize
import google.generativeai as genai
from tqdm.auto import tqdm
import os, time
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS


In [ ]:
def clean(df):
    for i in range(len(df)):
        sentence = df.loc[i , 'arabic_caption']
        sentence = normalize_unicode(sentence)
        sentence = dediac_ar(sentence)
        sentence = normalize_alef_maksura_ar(sentence)
        sentence = normalize_alef_ar(sentence)
        df.loc[i , 'arabic_caption'] = sentence
        #sentence = normalize_teh_marbura_ar
    return df  

In [ ]:
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embedding_model = HuggingFaceEmbeddings(model_name=model_name)

/tmp/ipykernel_36/450441757.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=model_name)
2025-08-18 02:44:30.693120: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755485071.019286      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755485071.105541      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
truth = pd.read_csv("/kaggle/input/dataset/Augmented_data.csv")
texts = truth['Description'].to_list() 
metadatas = [{"caption_index": i} for i in range(len(texts))]

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

vectorstore = FAISS.from_texts(texts, embedding_model, metadatas=metadatas)
vectorstore.save_local("new_faiss_index")

In [ ]:
loaded_vectorstore = FAISS.load_local("/kaggle/input/new-fiass", embedding_model, allow_dangerous_deserialization=True)

df = pd.read_csv("/kaggle/input/dataset/captions_finetuned_cleaned.csv" , encoding = 'utf-8-sig')
query = df.loc[0 , "caption"]  
results = loaded_vectorstore.similarity_search(query, k=3)

for i, result in enumerate(results):
    print(f"Result {i+1}:")
    print(f"Content: {result.page_content}")
    print(f"Caption Index: {result.metadata['caption_index']}")
    #print("-" * 50)

Result 1:
Content: تواصل هدم حاره المغاربه طريق النبي داود التي نزعت ملكيتها من اهلها وهدمت كجزء من تغيير مشهد البلده القديمه الذي قامت به قوات الاستعمار الصهيوني بعد حرب مجهوله التاريخ والمصور
Caption Index: 333
Result 2:
Content: تواصل هدم حاره المغاربه طريق النبي داود التي نزعت ملكيتها من اهلها وهدمت كجزء من تغيير مشهد البلده القديمه الذي قامت به قوات الاستعمار الصهيوني بعد حرب مجهوله التاريخ والمصور
Caption Index: 366
Result 3:
Content: اثار نسف دار و كوع في البلده القديمه في القدس ووفقا للتعقيب المرفق للصوره شمل النسف الدرج وجزء من العماره
Caption Index: 345


In [ ]:

os.environ['GEMINI_API_KEY'] = "Your_Api_Key"
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))


In [9]:
def RAG_Generated_caption(caption):
    query = caption
    retrieved_docs = loaded_vectorstore.similarity_search(query, k=5)

    retrieved_text = ""
    for i, result in enumerate(retrieved_docs):
        retrieved_text += f"- {result.page_content.strip()}\n"

    
    prompt = (
        "You are given an image caption generated by a model and a set of retrieved captions from a specialized dataset.\n"
        "Your task is to refine the generated caption to match the tone, style, and domain-specific language of the retrieved captions.\n"
        "Ensure historical accuracy and preserve the semantic meaning of the original caption.\n"
        "Match the tone, vocabulary, and structure used in the retrieved captions.\n"
        "Do not hallucinate facts. Focus on enhancing the alignment of the generated caption with the retrieved examples.\n\n"
        "Retrieved captions:\n"
        f"{retrieved_text}\n"
        "Original caption:\n"
        f"{query}\n\n"
        "Adapted caption:"
    )

    response = genai.GenerativeModel("gemini-2.5-flash").generate_content(prompt)
    reply = response.candidates[0].content.parts[0].text

    return reply


In [5]:
tokens = [
#all gemini tokens    
]

In [ ]:
import google.generativeai as genai
import os

for i, token in enumerate(tokens):
    try:
        os.environ['GEMINI_API_KEY'] = token
        genai.configure(api_key=token)
        model = genai.GenerativeModel('gemini-2.5-flash')
        response = model.generate_content("Hello, are you active?")
        print(f"Token {i} is VALID. Response: {response.text[:30]}...")
    except Exception as e:
        print(f"Token {i} is INVALID. Error: {e}")

#perhaps 465 and 466 were skipped, don't forget them 
#new_generated = []
#new_index = len(new_generated)
#1390 was skipped -> do it
j = 0
for i in range(1634, len(df)):
    try:
        new_cap = RAG_Generated_caption(df.loc[i, "arabic_caption"])
        file_name = df.loc[i, 'File Name']
        new_generated.append((file_name, new_cap))
        if i % 50 == 0:
            print(i)
        if (i - 465) % 200 == 0 and j < len(tokens): 
            os.environ['GEMINI_API_KEY'] = tokens[j]
            genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
            print("API token rotated BING BING")
            print(i)
            j += 1

    except Exception as e:
        print(f"Skipping index {i} due to error: {e}")
        continue

In [ ]:
df = pd.read_csv("/kaggle/input/dataset/captions_finetuned_cleaned_test.csv" , encoding = "utf-8-sig")

In [ ]:
df = pd.read_csv("/kaggle/input/dataset/captions_finetuned_cleaned_old.csv" , encoding = "utf-8-sig")
j = 0
useless = 0
gen = []

for i in tqdm(range(len(df)), desc="Generating captions", unit="file"):
    file_name = df.loc[i, 'image_id']
    caption = df.loc[i, "caption"]

    while True:  
        try:
            new_cap = RAG_Generated_caption(caption)
            gen.append((file_name, new_cap))
            break  

        except Exception as e:
            err_str = str(e).lower()

            if "quota" in err_str or "429" in err_str:
                print(f"Quota exceeded for token {j}, rotating...")
                j += 1
                if j >= len(tokens):
                    new_df = pd.DataFrame(gen, columns=["image_id", "caption"])
                    new_df.to_csv("/kaggle/working/captions_rag_test_clean.csv", encoding="utf-8-sig", index=False)
                    while True:
                        print(f"Still alive at {time.strftime('%Y-%m-%d %H:%M:%S')}")
                        time.sleep(14400)
                os.environ['GEMINI_API_KEY'] = tokens[j]
                genai.configure(api_key=tokens[j])
                time.sleep(2)  

            else:
                print(f"Unexpected error at index {i}: {e}. Retrying...")
                time.sleep(2)  


Generating captions:   1%|          | 4/752 [01:05<3:27:42, 16.66s/file]

Quota exceeded for token 0, rotating...


Generating captions:  15%|█▍        | 110/752 [28:27<2:44:50, 15.41s/file]

In [ ]:
new_df = pd.DataFrame(generated , columns = ["File Name" , "arabic_caption"])
new_df.to_csv("/kaggle/working/captions_rag_test_full.csv",encoding = "utf-8-sig")

In [ ]:
!zip -r /kaggle/working/new_faiss_index.zip /kaggle/working/new_faiss_index